In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np
import pandas_gbq

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,fbeta_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import pickle
from xgboost import XGBClassifier

In [2]:
# Load data from Google Biquery to local machine
project_id = 'dti-ds'
dataset_id = 'chris_dataset_002'
table_id = 'X_test_data_ecommerce_customer_churn'
region = 'us-central1'
bucket_name = 'chris_gcs_002'
blob_name = 'data/X_test_data_ecommerce_customer_churn.csv'

In [3]:
#load data from BQ
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()
df.head()

/home/christianmanan97/miniconda3/envs/christian-ai-env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,int64_field_0,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount
0,2104,0.0,13.0,4,Mobile,3,Married,2,1,7.0,123.66
1,2120,0.0,9.0,4,Mobile,1,Married,1,1,0.0,128.53
2,2018,0.0,21.0,3,Mobile,1,Single,2,0,0.0,128.71
3,700,0.0,13.0,3,Mobile,1,Married,1,0,1.0,125.23
4,2182,0.0,7.0,3,Mobile,1,Married,5,0,2.0,132.29


# Predictions

In [4]:
filename = "model.pkl"
model = pickle.load(open(filename,"rb"))

# # Evaluation
predictions = model.predict(df)
predictions

/home/christianmanan97/miniconda3/envs/christian-ai-env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:11:25] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


array([1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [5]:
X_iterative = pd.DataFrame({"Tenure":[1],
                        "WarehouseToHome":[50],
                        "NumberOfDeviceRegistered":[5],
                        "PreferedOrderCat":["Mobile"],
                        "SatisfactionScore":[5],
                        "MaritalStatus":["Single"],
                        "NumberOfAddress":[5],
                        "Complain":[1],
                        "DaySinceLastOrder":[50],
                        "CashbackAmount":[150]})

model.predict(X_iterative)

array([1])

In [6]:
X_iterative = pd.DataFrame({"Tenure":[100],
                        "WarehouseToHome":[50],
                        "NumberOfDeviceRegistered":[5],
                        "PreferedOrderCat":["Mobile"],
                        "SatisfactionScore":[5],
                        "MaritalStatus":["Single"],
                        "NumberOfAddress":[5],
                        "Complain":[1],
                        "DaySinceLastOrder":[50],
                        "CashbackAmount":[150]})

model.predict(X_iterative)

array([0])

# Exporting to bigquery

In [7]:
df = df.join(pd.DataFrame({"prediction":predictions.tolist()}))

In [8]:
df

,int64_field_0,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount,prediction
0,2104,0.0,13.0,4,Mobile,3,Married,2,1,7.0,123.66,1
1,2120,0.0,9.0,4,Mobile,1,Married,1,1,0.0,128.53,0
2,2018,0.0,21.0,3,Mobile,1,Single,2,0,0.0,128.71,1
3,700,0.0,13.0,3,Mobile,1,Married,1,0,1.0,125.23,0
4,2182,0.0,7.0,3,Mobile,1,Married,5,0,2.0,132.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...
649,3099,1.0,34.0,4,Laptop & Accessory,1,Single,3,1,8.0,183.13,1
650,1019,1.0,8.0,3,Laptop & Accessory,4,Married,5,0,4.0,148.55,1
651,3654,1.0,35.0,4,Laptop & Accessory,4,Divorced,3,0,5.0,187.72,0
652,2318,1.0,9.0,4,Laptop & Accessory,1,Married,2,0,3.0,167.27,0


# Export dataframe to Bigquerry table

In [9]:
project_id = 'dti-ds'
dataset_id = 'chris_dataset_002'
table_id = 'X_test_with_prediction'
table_full_id = f'{project_id}.{dataset_id}.{table_id}'
pandas_gbq.to_gbq(df, table_full_id, project_id=project_id, if_exists='replace')